### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

In [ ]:
%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.45.4

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from transformers import BitsAndBytesConfig

model_name = 'Enoch/llama-7b-hf'

# Конфигурация для 4-битной квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

# Загрузка токенизатора
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Загрузка модели с квантизацией
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Передаем конфиг квантизации
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32
)

# Отключаем вычисление градиентов
for param in model.parameters():
    param.requires_grad = False

# Настройка gradient checkpointing
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Prompt tuning: the story of a fox

![img](https://i.imgur.com/Ux3qQAu.png)

In [3]:
prompt = 'A quick brown fox'

# Токенизация промта и подготовка входных данных для модели
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(10):  # Генерируем 10 новых токенов
    # Получаем логиты (raw predictions) для следующего токена
    logits = model(**batch).logits

    # Выбираем токен с наибольшей вероятностью (жадная стратегия)
    next_token = logits[0, -1].argmax(-1).reshape(1, 1)

    # Добавляем новый токен к существующей последовательности
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)

    # Обновляем маску внимания (добавляем 1 для нового токена)
    batch['attention_mask'] = torch.cat(
        [batch['attention_mask'],
        torch.ones_like(next_token)  # Создаем маску из единиц той же формы
    ], dim=-1)

# Декодируем и выводим сгенерированный текст
print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output: <s>A quick brown fox jumps over the lazy dog.
A quick


What a blatant lie! This particular fox assures you that it didn't in fact jump over the lazy dog. No, sir! The fox was just minding its own business. __Your task is to train the model to say truth: no dog was jumped over today.__

In [5]:
# Истинное предложение, которое мы будем использовать для вычисления потерь
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"

# Токенизация текста и подготовка входных данных для модели
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

# Пропускаем текст через модель для получения предсказаний
outputs = model(**batch)

# Вычисляем логиты для предсказания следующего слова
next_word_logits = outputs.logits[:, :-1]

# Получаем правильные следующие токены (сдвигаем входные данные на 1 вправо)
true_next_tokens = batch['input_ids'][:, 1:]

# Вычисляем функцию потерь (cross-entropy)
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print("Loss:", loss.item())

Loss: 3.0728819370269775


Except, we can't train the entire model - that would be 28GB gradients in float32. Instead, let's run [prompt tuning](https://arxiv.org/abs/2104.08691).

![img](https://i.imgur.com/VwNNKnb.png)


In [6]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
    """
    Для настройки промптов (prompt tuning) необходимо заменить оригинальные эмбеддинги модели
    на этот слой, который вставляет обучаемые промпты вместо первых N токенов."""

    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings  # Оригинальный слой эмбеддингов модели
        self.num_prompts = num_prompts  # Количество обучаемых промптов

        # Инициализируем обучаемые промпты (случайные значения)
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim),
            requires_grad=True)  # Параметры будут обучаться

    def forward(self, input_ids: torch.LongTensor):
        # input_ids shape: [размер_батча, длина_последовательности]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts, "Последовательность должна быть длиннее числа промптов"
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), \
            "Не забудьте добавить BOS-токены в начало input_ids"

        # Задача: заменить первые num_prompts токенов на обучаемые промпты
        # Мы предполагаем, что в начале последовательности добавлены num_prompts pad-токенов

        # Результат должен содержать эмбеддинги для всех токенов, где:
        # - Первые num_prompts векторов - это learnable_prompts
        # - Остальные вектора - обычные эмбеддинги слов
        # Важно: используйте torch.cat вместо присваивания, так как работаем с обучаемыми параметрами

        # Получаем оригинальные эмбеддинги
        raw_embs = self.original_word_embeddings(input_ids)

        # Объединяем обучаемые промпты с остальными эмбеддингами
        embs = torch.cat([self.learnable_prompts, raw_embs[:, self.num_prompts:]], dim=1)

        return embs

In [7]:
num_prompts = 16

# Инициализируем слой эмбеддингов с промптами (заменяет стандартные эмбеддинги модели)
test_emb_layer = WordEmbeddingsWithLearnedPrompts(
    model.model.embed_tokens,  # Оригинальный слой эмбеддингов модели
    num_prompts=num_prompts
).to(device)

test_input_ids = tokenizer("a cat say on a may", return_tensors='pt')['input_ids'].to(device)

# Создаем "пространство" для промптов в начале последовательности:
# заполняем pad_token_id (обычно 0 или специальный токен)
space_for_prompts = torch.full(
    [len(test_input_ids), num_prompts],
    fill_value=tokenizer.pad_token_id,
    dtype=torch.int64,
    device=device
)

# Объединяем промпты с исходными токенами
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.autocast('cuda'):
    test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

# Проверяем корректность работы:
# 1. Совпадение размерностей первых двух осей
assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
# 2. Размерность эмбеддингов соответствует скрытому размеру модели
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
# 3. Первые num_prompts эмбеддингов соответствуют обучаемым промптам
assert torch.allclose(test_prompt_embeddings[:, :num_prompts], test_emb_layer.learnable_prompts.float())
# 4. Остальные эмбеддинги соответствуют оригинальным эмбеддингам модели
assert torch.allclose(test_prompt_embeddings[:, num_prompts:], model.model.embed_tokens(test_input_ids).float())

print("Проверки пройдены! Всё работает корректно!")

Проверки пройдены! Всё работает корректно!


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [9]:
# Проверяем, что слой эмбеддингов модели является стандартным nn.Embedding
assert isinstance(model.model.embed_tokens, nn.Embedding), (
    "Слой эмбеддингов уже был заменен. Если замена работает некорректно, "
    "перезагрузите модель снова"
)

# Заменяем стандартный слой эмбеддингов на наш кастомный слой с обучаемыми промптами
model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(
    model.model.embed_tokens,
    num_prompts=num_prompts
).to(device)

opt = torch.optim.Adam(
    [model.model.embed_tokens.learnable_prompts],  # Только обучаемые промпты
    lr=0.01
)

In [10]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"

batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

# Создаем пространство для промптов в начале последовательности
# Заполняем pad_token_id (обычно 0 или специальный токен)
space_for_prompts = torch.full(
    [len(batch['input_ids']), num_prompts],
    fill_value=tokenizer.pad_token_id,
    dtype=torch.int64,
    device=device
)

# Модифицируем входные данные:
# Добавляем промпты перед исходными токенами
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
# Обновляем маску внимания (1 для промптов, исходная маска для остальных токенов)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(100):
    outputs = model(**batch)

    # Вычисляем логиты для предсказания следующих токенов
    # Игнорируем первые num_prompts и последний токен
    next_word_logits = outputs.logits[:, num_prompts:-1, :]

    # Получаем правильные следующие токены
    true_next_tokens = batch['input_ids'][:, num_prompts+1:]

    # Вычисляем функцию потерь (cross-entropy)
    loss = F.cross_entropy(
        next_word_logits.flatten(0, 1),  # Преобразуем в [batch_size*seq_len, vocab_size]
        true_next_tokens.flatten(0, 1)   # Преобразуем в [batch_size*seq_len]
    )

    if i % 10 == 0:
        print(f"Step {i}, Loss: {loss.item():.4f}")

    opt.zero_grad()
    loss.backward()
    opt.step()

assert loss.item() <= 0.1, f"Обучение не удалось, loss: {loss.item():.4f}"
print("Отличная работа! Промпты успешно обучены!")

Step 0, Loss: 7.3483
Step 10, Loss: 3.6654
Step 20, Loss: 1.8783
Step 30, Loss: 0.6268
Step 40, Loss: 0.0943
Step 50, Loss: 0.0303
Step 60, Loss: 0.0155
Step 70, Loss: 0.0101
Step 80, Loss: 0.0077
Step 90, Loss: 0.0064
Отличная работа! Промпты успешно обучены!


In [11]:
prompt = 'A quick brown fox'

batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

# Добавляем обученные промпты в начало последовательности:
# 1. Заменяем input_ids на [промпты + оригинальные токены]
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
# 2. Обновляем маску внимания (1 для промптов и оригинальных токенов)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(15):
    # Получаем следующий токен (жадное декодирование)
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)

    # Добавляем новый токен к последовательности
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)

    # Обновляем маску внимания для нового токена
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

# [num_prompts:] - игнорируем первые num_prompts токенов (это наши промпты)
print("\nOutput:", tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist()))


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Using HuggingFace PEFT

[`peft`](https://huggingface.co/docs/peft/index) is a transformer's sister library that allows you to apply various __p__arameter __e__fficient __f__ine-__t__uning methods to pre-trained transformers. The library imlements both prompt tuning, prefix tuning, as well as several adapter-based techniques under a common interface:



In [4]:
import peft

# Проверяем, что слой embed_tokens модели является объектом nn.Embedding, иначе нужно перезагрузить модель
assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"

# Создаем конфигурацию для настройки подсказок (Prompt Tuning) с указанием типа задачи и количества виртуальных токенов
peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=16)

# Применяем конфигурацию PEFT к модели, добавляя возможность тонкой настройки (модель изменяется на месте)
model = peft.get_peft_model(model, peft_config)  # примечание: для большинства методов PEFT эта строка также изменяет модель напрямую

print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 65536
Total parameters (excluding quantization): 3500478464


In [5]:
# Ваша задача: оптимизировать модель, обёрнутую в PEFT, чтобы достичь значения ошибки предсказания следующего токена < 0.1, используя PEFT
# Обратите внимание: больше не нужно добавлять PAD-токены в начало, но всё ещё нужно пропустить :num_virtual_tokens: первых логитов.
# Наконец, сгенерируйте предложение, чтобы убедиться, что модель выучила правду.

opt = torch.optim.Adam(model.parameters(), lr=0.01)

the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"

batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

# Устанавливаем количество виртуальных токенов (подсказок), используемых в PEFT
num_prompts = 16


for i in range(100):
    outputs = model(**batch)

    # Извлекаем логиты для предсказания следующего слова, пропуская первые num_prompts токенов и последний
    next_word_logits = outputs.logits[:, num_prompts:-1, :]

    true_next_tokens = batch['input_ids'][:, 1:]

    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

    if i % 10 == 0:
        print("Loss:", loss.item())

    opt.zero_grad()
    loss.backward()
    opt.step()

assert loss.item() <= 0.1
print("Good job!")

Loss: 7.261453151702881
Loss: 4.480815410614014
Loss: 3.0683929920196533
Loss: 1.851556658744812
Loss: 0.7476693987846375
Loss: 0.22174982726573944
Loss: 0.07413368672132492
Loss: 0.032651662826538086
Loss: 0.01911357417702675
Loss: 0.013293243013322353
Good job!


In [6]:
# Задаём начальный промпт для генерации текста
prompt = 'A quick brown fox'

batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(15):
    # Получаем логиты для последнего токена в последовательности и выбираем токен с максимальной вероятностью
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)

    # Добавляем новый токен к последовательности input_ids
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)

    # Обновляем маску внимания, добавляя 1 для нового токена
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Parameter-efficient finetuning with LoRA

When training on more serious tasks, you can use low-rank adapters based on the [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf).

The core idea is to add low-rank adapters __in parallel with existing linear layers,__ like this:
<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In the original LoRA paper, the adapters were only added to attention projection matrices. However, [subsequent works](https://arxiv.org/abs/2305.14314) show that it is useful to adapt FFNs as well. But before we do any training, we need to implement the basic LoRA layer.

In [7]:
# Перезагружаем модель, чтобы удалить все предыдущие настройки PEFT
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.float32,
    # Веса в 4 битах, а нормализация слоёв и активации в fp32
)

# Отключаем возможность обучения всех параметров модели (замораживаем веса)
for param in model.parameters():
    param.requires_grad = False

# Включаем сохранение промежуточных градиентов для экономии памяти во время обучения
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [8]:
class LoRALayer(nn.Module):
    """Оборачивает линейный слой адаптером в стиле LoRA. Работает с существующим линейным слоем OPT"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # Предобученный (замороженный) линейный слой
        # Создаём параметр adapter_A — матрицу размера (входные_признаки, ранг)
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))

        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        # Создаём параметр adapter_B — матрицу размера (ранг, выходные_признаки), изначально заполненную нулями
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Применяем self.module и адаптер LoRA, возвращаем сумму (выходы модуля + выходы адаптера)
        # Вычисляем изменение весов как произведение матриц adapter_A и adapter_B
        delta_W = self.adapter_A @ self.adapter_B
        # Вычисляем выход исходного модуля и добавляем к нему результат адаптера
        output = self.module(input) + torch.matmul(input, delta_W)
        return output

In [9]:
# Тестируем вашу реализацию
# Создаём тестовый линейный слой с размерностью 128x128
test_linear = nn.Linear(128, 128)
# Заполняем веса единичной матрицей (диагональ из единиц)
test_linear.weight.data[...] = torch.eye(128)
# Создаём тестовый адаптер LoRA с рангом 8, оборачивая линейный слой
test_adapter = LoRALayer(test_linear, rank=8)

# Проверяем, что прямой проход работает корректно: выход адаптера должен быть равен смещению + 1
assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "проверьте ваш прямой проход"

# Заполняем матрицу adapter_A значениями от 0.1 до -0.5, преобразуя в форму (128, 8)
test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
# Заполняем матрицу adapter_B значениями от 0.5 до -0.1, преобразуя в форму (8, 128)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
# Заполняем смещение линейного слоя значениями от 1.0 до -1.0
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

# Вычисляем фиктивную потерю как среднеквадратичную ошибку между выходом адаптера и целевыми значениями
dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
# Проверяем, что значение потери близко к ожидаемому (1.3711389) с допустимой погрешностью
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)

# Выполняем обратное распространение ошибки
dummy_loss.backward()
# Проверяем, что градиенты есть у всех весов адаптера (A и B)
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "у некоторых весов адаптера нет градиента"
# Проверяем сумму градиентов для adapter_A, она должна быть близка к -0.60158
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "неверный градиент для A"
# Проверяем сумму градиентов для adapter_B, она должна быть близка к 0.9931
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "неверный градиент для B"
# Примечание: неверный градиент означает, что ваш код отличается от статьи LoRA ИЛИ не совместим с autograd (например, используется no_grad)

# Удаляем временные переменные, чтобы освободить память
del dummy_loss, test_linear, test_adapter
# Выводим сообщение об успешном прохождении всех тестов
print("Все тесты пройдены!")

Все тесты пройдены!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [11]:
# Устанавливаем ранг для LoRA-адаптера
lora_rank = 8

for name, module in model.model.layers.named_modules():
    # Проверяем, является ли модуль слоем декодера Llama (LlamaDecoderLayer)
    if 'LlamaDecoderLayer' in repr(type(module)):
        # Заменяем q_proj, k_proj, v_proj на LoRA-слой с заданным рангом
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

# Проверяем, что общее количество LoRA-слоёв в модели равно 96
assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # для Llama-7B

In [12]:
# Токенизируем входной текст и преобразуем его в тензоры PyTorch, исключая token_type_ids
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)

for key in batch:
    batch[key] = batch[key].cuda()

# Тестируем один шаг обучения, чтобы убедиться, что получаем осмысленные градиенты
with torch.autocast('cuda', dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()


for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        # Убеждаемся, что градиент для adapter_B существует
        assert module.adapter_B.grad is not None
        # Проверяем, что норма градиента adapter_B больше нуля (градиенты осмысленные)
        assert module.adapter_B.grad.norm().item() > 0

# Сбрасываем градиенты модели, устанавливая их в None для экономии памяти
model.zero_grad(set_to_none=True)
print("Проверка градиентов прошла успешно, отлично сделано!")

Проверка градиентов прошла успешно, отлично сделано!


### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [13]:
import datasets

# Загружаем датасет с английскими цитатами, берём только первые 32 строки
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]")  # 32 строки
# Применяем токенизацию к столбцу 'quote' в датасете, обрабатывая данные батчами
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

model._hf_peft_config_loaded = True  # убираем предупреждение от HF trainer


trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # Примечание: если нужен больший размер батча, увеличьте gradient_accumulation_steps
        warmup_steps=250,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        report_to=['tensorboard']
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# Если появляются предупреждения о кэше, установите `model.config.use_cache = False`, чтобы их убрать. Включите обратно для вывода!

trainer.train()

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.891500
2,1.696000
3,0.893900
4,1.746100
5,1.174900
6,0.734100
7,1.544300
8,1.091300
9,0.677000
10,1.448700


TrainOutput(global_step=100, training_loss=0.545358849875629, metrics={'train_runtime': 42.7212, 'train_samples_per_second': 4.682, 'train_steps_per_second': 2.341, 'total_flos': 621258424123392.0, 'train_loss': 0.545358849875629, 'epoch': 6.25})